## SVM linear, SVM RBF, Adaboost, XGboost（只用降維後純文本做預測）

### 載入套件

In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from time import time
from sklearn import metrics
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

### 讀檔

In [2]:
## stock sign
filePath = "./preprocess/tokens+stockSign/"
df_stockSign_covid = pd.read_csv(filePath + "cvaw_covid_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]
df_stockSign_stock = pd.read_csv(filePath + "cvaw_stock_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]

### 函式：讀 tokens data，切成 x_train, x_test

In [3]:
def SVD_embedding(tokens_name):
    df_embedding = pd.read_csv("./TFIDF/" + tokens_name + "_stockSign_TFIDF_TruncatedSVD.csv")
    
    df_embedding = df_embedding.iloc[:, 1:501]
    
    x_train, x_test = train_test_split(df_embedding, test_size=0.20, random_state=404)
    
    return x_train, x_test

In [4]:
def word2vec_embedding(tokens_name):
    df_embedding = pd.read_csv("./Word2Vec/" + tokens_name + "_stockSign_word2vec.csv")
    
    df_embedding = df_embedding.iloc[:, 1:501]
    
    x_train, x_test = train_test_split(df_embedding, test_size=0.20, random_state=404)
    
    return x_train, x_test

### 口罩類股 斷詞 testing & training split

In [5]:
x_train_mask, x_test_mask = word2vec_embedding("covid_token")
y_train_mask, y_test_mask = train_test_split(df_stockSign_covid[["stockRise_mask"]], test_size=0.20, random_state=404)

In [6]:
x_train_mask

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
7271,0.466977,0.490552,-0.046918,-0.202188,0.166707,0.052072,0.100822,0.127350,0.293276,0.144377,...,0.046730,0.167719,-0.160782,0.080226,-0.125111,-0.088842,0.191329,-0.090425,0.034194,-0.061873
317,-0.137972,0.238994,-0.103087,-0.276188,-0.096632,0.055066,-0.054771,0.103535,-0.027499,0.190077,...,0.052938,0.030108,0.247392,-0.107904,-0.122517,-0.083494,0.085071,0.077354,0.072515,0.231514
1925,-0.272390,-0.219359,0.030242,0.378799,0.014876,-0.015882,-0.218432,0.099289,0.055401,0.247769,...,-0.082648,-0.125051,-0.139779,-0.022874,0.055691,-0.276878,0.057967,0.295714,-0.024754,0.151208
3188,-0.165721,0.238822,-0.049381,-0.021363,0.317870,-0.113444,-0.313132,0.128785,0.005647,0.069155,...,0.219933,0.324596,-0.353152,0.234333,-0.128627,0.060161,0.052233,-0.132020,0.074416,0.239536
7289,-0.231800,-0.203382,0.166360,-0.131830,0.190765,-0.032649,0.228955,-0.392035,-0.171504,0.106874,...,0.144791,0.023806,0.343031,-0.069726,-0.126730,0.170638,0.217938,0.494262,0.491883,0.135065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.635976,0.370562,-0.194026,-0.273566,0.023036,0.088587,0.273419,0.192486,0.337552,0.173713,...,0.366876,-0.402000,-0.139996,-0.355655,-0.181885,-0.530296,-0.050131,0.032169,-0.168377,-0.005768
5108,-0.282629,-0.084010,-0.158663,0.005067,-0.008301,-0.057799,-0.283823,-0.172922,-0.028213,0.138294,...,-0.053598,-0.079068,0.208914,-0.076541,-0.203598,-0.086399,-0.062242,-0.176672,-0.209389,0.280768
5302,-0.183439,-0.147837,-0.012782,0.516806,-0.024616,-0.445040,0.228963,-0.249978,-0.056679,0.074657,...,-0.059065,0.019059,0.297540,-0.307480,-0.170445,-0.036629,-0.077072,0.211082,-0.150104,0.504225
5994,0.219891,0.571944,0.013836,-0.294966,0.212134,0.060913,0.161956,-0.021638,0.250821,-0.284589,...,-0.100324,-0.328688,-0.115157,0.037604,0.034764,-0.154887,0.087008,-0.282653,-0.279951,-0.046932


In [7]:
x_train_mask = x_train_mask.values.tolist()
x_test_mask = x_test_mask.values.tolist()
y_train_mask = y_train_mask["stockRise_mask"].tolist()
y_test_mask = y_test_mask["stockRise_mask"].tolist()

### 檢測試劑類股 斷詞 testing & training split

In [8]:
x_train_testKits, x_test_testKits = SVD_embedding("stock_token")
y_train_testKits, y_test_testKits = train_test_split(df_stockSign_stock[["stockRise_testKits"]], test_size=0.20, random_state=404)

In [9]:
x_train_testKits

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
6004,0.110194,-0.056617,0.016188,0.051068,0.024915,-0.024274,-0.014723,-0.026258,-0.055791,-0.005208,...,0.006400,-0.009511,-0.008505,0.003994,0.017639,-0.015786,0.002318,0.026095,0.000526,0.005658
6702,0.215222,-0.029807,-0.106440,-0.060623,-0.081690,-0.008828,-0.005497,0.125946,-0.049947,-0.023866,...,0.000037,0.008915,0.006572,-0.016545,-0.014984,0.025858,-0.018918,-0.001705,0.016183,0.027335
4971,0.190443,0.456539,-0.089589,0.113550,0.085209,-0.011482,-0.013740,0.040625,-0.076117,0.018417,...,0.045564,0.011609,0.001865,-0.018082,0.016909,0.012102,0.003540,-0.022592,0.000633,-0.007836
1867,0.230840,-0.021076,-0.092870,-0.061819,-0.110322,-0.008187,0.002349,0.030462,0.027457,0.002368,...,-0.004193,0.005546,0.020345,0.010719,-0.002002,0.002424,0.010414,-0.025819,0.006383,0.013708
2484,0.153676,-0.052038,0.060150,0.017483,0.192235,0.037284,-0.009671,0.023961,0.167582,-0.376188,...,0.009907,0.001814,-0.013591,0.012238,-0.005709,0.024119,-0.035278,0.012393,-0.016911,0.011092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.114674,0.005346,-0.026531,-0.063207,0.020567,0.034371,0.034781,-0.019504,-0.047736,0.029909,...,-0.014838,-0.003390,0.016402,-0.007536,0.015456,0.012212,0.022440,0.019762,-0.006434,-0.001645
5108,0.140593,-0.014271,-0.065399,-0.031681,-0.076528,-0.009261,0.001531,0.053130,-0.032995,-0.036033,...,0.006875,-0.007357,-0.017490,0.045533,0.011390,0.017935,0.011710,0.002624,-0.019156,-0.023624
5302,0.080773,-0.011286,0.018688,0.016437,-0.017477,-0.009216,-0.000077,-0.032430,0.000492,0.047174,...,0.018001,-0.024973,-0.013130,0.004887,-0.012008,0.003088,0.024134,0.012363,-0.005537,0.018533
5884,0.158278,0.128662,-0.078809,0.024062,0.000665,-0.018054,0.009116,0.010543,0.001331,-0.067082,...,0.005738,0.001595,-0.016617,0.008776,-0.010579,0.002256,-0.008569,0.012491,-0.017333,-0.002944


In [10]:
x_train_testKits = x_train_testKits.values.tolist()
x_test_testKits = x_test_testKits.values.tolist()
y_train_testKits = y_train_testKits["stockRise_testKits"].tolist()
y_test_testKits = y_test_testKits["stockRise_testKits"].tolist()

### 疫苗類股 斷詞 testing & training split

In [11]:
x_train_vaccine, x_test_vaccine = SVD_embedding("stock_token")
y_train_vaccine, y_test_vaccine = train_test_split(df_stockSign_stock[["stockRise_vaccine"]], test_size=0.20, random_state=404)

In [12]:
x_train_vaccine = x_train_vaccine.values.tolist()
x_test_vaccine = x_test_vaccine.values.tolist()
y_train_vaccine = y_train_vaccine["stockRise_vaccine"].tolist()
y_test_vaccine = y_test_vaccine["stockRise_vaccine"].tolist()

### SVM Model 函式

In [13]:
def tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, Linear, embedding):
    
    if Linear == True:
        SVM_model = SVC(kernel = "linear", C = 1.0, probability=True)
    else:
        SVM_model = SVC(kernel = "rbf", gamma = "scale", C = 1.0, probability=True)
    
    t0 = time()
    SVM_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(SVM_model.predict(x_test))
    
    if Linear == True:
        print(tokens_name + " predict " + predict_stocks_name + "(SVM Linear + " + embedding + ")")
    else:
        print(tokens_name + " predict " + predict_stocks_name + "(SVM RBF + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

### adaboost model 函式

In [26]:
def adaboost_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding, n_estimator):
    
    adaboost_model = AdaBoostClassifier(n_estimators = n_estimator, random_state = 404)
    
    t0 = time()
    adaboost_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(adaboost_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(adaboost + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

### xgboost model 函式（train, test 不能轉為 list）

In [27]:
def xgboost_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding, stock_type):
    
    y_test = y_test["stockRise_" + stock_type].tolist()
    
    xgboost_model = XGBClassifier(random_state = 404)
    
    t0 = time()
    xgboost_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(xgboost_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(xgboost + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

### Random forest 函式

In [16]:
def randomForest_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding):
    
    randomForest_model = RandomForestClassifier(n_estimators=500, max_features=50, random_state=404)
    
    t0 = time()
    randomForest_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(randomForest_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(random forest + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

### 預測口罩類股漲跌效果（word2vec covid 板 ckip 斷詞）

In [14]:
tokens_predict_stocks(x_train_mask, y_train_mask, x_test_mask, y_test_mask, "Covid CKIP token", "mask stock", True, "word2vec")

done in 171.444s.
Covid CKIP token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.40      0.49      0.44       557
           0       0.00      0.00      0.00       324
           1       0.44      0.58      0.50       622

    accuracy                           0.42      1503
   macro avg       0.28      0.36      0.31      1503
weighted avg       0.33      0.42      0.37      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
tokens_predict_stocks(x_train_mask, y_train_mask, x_test_mask, y_test_mask, "Covid CKIP token", "mask stock", False, "word2vec")

done in 148.992s.
Covid CKIP token predict mask stock(SVM RBF + word2vec)
              precision    recall  f1-score   support

          -1       0.41      0.54      0.46       557
           0       0.29      0.01      0.01       324
           1       0.44      0.54      0.49       622

    accuracy                           0.42      1503
   macro avg       0.38      0.36      0.32      1503
weighted avg       0.40      0.42      0.38      1503



In [28]:
adaboost_tokens_predict_stocks(x_train_mask, y_train_mask, x_test_mask, y_test_mask, "Covid CKIP token", "mask stock", "word2vec", 100)

done in 36.189s.
Covid CKIP token predict mask stock(adaboost + word2vec)
              precision    recall  f1-score   support

          -1       0.37      0.44      0.40       557
           0       0.30      0.17      0.22       324
           1       0.41      0.43      0.42       622

    accuracy                           0.38      1503
   macro avg       0.36      0.35      0.35      1503
weighted avg       0.37      0.38      0.37      1503



In [36]:
xgboost_tokens_predict_stocks(x_train_mask, y_train_mask, x_test_mask, y_test_mask, "Covid CKIP token", "mask stock", "word2vec", "mask")

C:\Users\molly\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\molly\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[15:35:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 51.570s.
Covid CKIP token predict mask stock(xgboost + word2vec)
              precision    recall  f1-score   support

          -1       0.40      0.45      0.42       557
           0       0.26      0.13      0.17       324
           1       0.44      0.49      0.46       622

    accuracy                           0.40      1503
   macro avg       0.36      0.36      0.35      1503
weighted avg       0.38      0.40      0.38      1503



In [17]:
randomForest_tokens_predict_stocks(x_train_mask, y_train_mask, x_test_mask, y_test_mask, "Covid CKIP token", "mask stock", "word2vec")

done in 113.179s.
Covid CKIP token predict mask stock(random forest + word2vec)
              precision    recall  f1-score   support

          -1       0.39      0.46      0.42       557
           0       0.28      0.05      0.08       324
           1       0.44      0.56      0.49       622

    accuracy                           0.41      1503
   macro avg       0.37      0.36      0.33      1503
weighted avg       0.39      0.41      0.38      1503



### 預測檢測試劑類股漲跌效果（SVD stock 板 jieba 斷詞）

In [22]:
tokens_predict_stocks(x_train_testKits, y_train_testKits, x_test_testKits, y_test_testKits, "Stock jieba token", "testKits stock", True, "SVD vector")

done in 223.275s.
Stock jieba token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.44      0.06      0.10       525
           0       0.45      0.95      0.61       855
           1       0.33      0.03      0.06       566

    accuracy                           0.44      1946
   macro avg       0.40      0.35      0.26      1946
weighted avg       0.41      0.44      0.31      1946



In [23]:
tokens_predict_stocks(x_train_testKits, y_train_testKits, x_test_testKits, y_test_testKits, "Stock jieba token", "testKits stock", False, "SVD vector")

done in 243.376s.
Stock jieba token predict testKits stock(SVM RBF + SVD vector)
              precision    recall  f1-score   support

          -1       0.40      0.16      0.23       525
           0       0.46      0.81      0.59       855
           1       0.36      0.14      0.20       566

    accuracy                           0.44      1946
   macro avg       0.40      0.37      0.34      1946
weighted avg       0.41      0.44      0.38      1946



In [29]:
adaboost_tokens_predict_stocks(x_train_testKits, y_train_testKits, x_test_testKits, y_test_testKits, "Stock jieba token", "testKits stock", "SVD vector", 100)

done in 45.738s.
Stock jieba token predict testKits stock(adaboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.30      0.24      0.27       525
           0       0.46      0.63      0.53       855
           1       0.31      0.21      0.25       566

    accuracy                           0.40      1946
   macro avg       0.36      0.36      0.35      1946
weighted avg       0.38      0.40      0.38      1946



In [37]:
xgboost_tokens_predict_stocks(x_train_testKits, y_train_testKits, x_test_testKits, y_test_testKits, "Stock jieba token", "testKits stock", "SVD vector", "testKits")

[15:36:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 81.634s.
Stock jieba token predict testKits stock(xgboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.31      0.20      0.24       525
           0       0.45      0.64      0.53       855
           1       0.33      0.23      0.27       566

    accuracy                           0.40      1946
   macro avg       0.36      0.36      0.35      1946
weighted avg       0.38      0.40      0.38      1946



In [18]:
randomForest_tokens_predict_stocks(x_train_testKits, y_train_testKits, x_test_testKits, y_test_testKits, "Stock jieba token", "testKits stock", "SVD vector")

done in 159.724s.
Stock jieba token predict testKits stock(random forest + SVD vector)
              precision    recall  f1-score   support

          -1       0.42      0.05      0.09       525
           0       0.44      0.95      0.60       855
           1       0.31      0.03      0.06       566

    accuracy                           0.44      1946
   macro avg       0.39      0.34      0.25      1946
weighted avg       0.40      0.44      0.31      1946



### 預測疫苗類股漲跌效果（SVD stock 板 jieba 斷詞）

In [24]:
tokens_predict_stocks(x_train_vaccine, y_train_vaccine, x_test_vaccine, y_test_vaccine, "Stock jieba token", "vaccine stock", True, "SVD vector")

done in 197.480s.
Stock jieba token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.12      0.00      0.01       428
           0       0.56      0.99      0.72      1082
           1       0.75      0.01      0.01       436

    accuracy                           0.56      1946
   macro avg       0.48      0.34      0.25      1946
weighted avg       0.51      0.56      0.40      1946



In [25]:
tokens_predict_stocks(x_train_vaccine, y_train_vaccine, x_test_vaccine, y_test_vaccine, "Stock jieba token", "vaccine stock", False, "SVD vector")

done in 246.035s.
Stock jieba token predict vaccine stock(SVM RBF + SVD vector)
              precision    recall  f1-score   support

          -1       0.35      0.05      0.08       428
           0       0.57      0.98      0.72      1082
           1       0.34      0.03      0.06       436

    accuracy                           0.56      1946
   macro avg       0.42      0.35      0.29      1946
weighted avg       0.47      0.56      0.43      1946



In [30]:
adaboost_tokens_predict_stocks(x_train_vaccine, y_train_vaccine, x_test_vaccine, y_test_vaccine, "Stock jieba token", "vaccine stock", "SVD vector", 100)

done in 42.233s.
Stock jieba token predict vaccine stock(adaboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.23      0.08      0.12       428
           0       0.57      0.86      0.69      1082
           1       0.25      0.09      0.13       436

    accuracy                           0.52      1946
   macro avg       0.35      0.34      0.31      1946
weighted avg       0.42      0.52      0.44      1946



In [38]:
xgboost_tokens_predict_stocks(x_train_vaccine, y_train_vaccine, x_test_vaccine, y_test_vaccine, "Stock jieba token", "vaccine stock", "SVD vector", "vaccine")

[15:37:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 84.425s.
Stock jieba token predict vaccine stock(xgboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.36      0.12      0.18       428
           0       0.59      0.91      0.72      1082
           1       0.34      0.10      0.16       436

    accuracy                           0.56      1946
   macro avg       0.43      0.38      0.35      1946
weighted avg       0.48      0.56      0.47      1946



In [19]:
randomForest_tokens_predict_stocks(x_train_vaccine, y_train_vaccine, x_test_vaccine, y_test_vaccine, "Stock jieba token", "vaccine stock", "SVD vector")

done in 157.701s.
Stock jieba token predict vaccine stock(random forest + SVD vector)
              precision    recall  f1-score   support

          -1       0.47      0.02      0.04       428
           0       0.56      1.00      0.72      1082
           1       0.60      0.02      0.04       436

    accuracy                           0.56      1946
   macro avg       0.55      0.35      0.27      1946
weighted avg       0.55      0.56      0.42      1946

